<a href="https://colab.research.google.com/github/rodr1ggoql17/Procesamiento-Lenguaje-Natural/blob/main/procesos_de_markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a los modelos de Markov en PLN

Andrey Markov desarrollo una teoría que con el tiempo los sistemas cambian con el tiempo (modelos de Markov)

# Donde aplicamos el modelo


*   Finanzas, para predecir los altos y bajos del mercado
*   Aprendizaje por refuerzo
*   Modelos ocultos de markov
*   Cadena de Markov Monte Carlo (MCMC), rastrear la evolucion genetica



# Propiedad fundamental de los Modelos de Markov

*   Si voy a pasar de un estado a otro, no importan los antecedentes


# Estructura y entrenamiento de un modelo de Markov

*  Definición y componentes del modelo
*  Matriz de transición de estados
*  Entrenamiento del modelo

Resumen: Estoy en un estado, ¿cual es la probabilidad de que pase a otro estado?




#Ejemplo
Construcción de un generador de texto / clasificador de texto




In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -U scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [ ]:
archivos = [
    '/content/drive/MyDrive/CURSO NLP/data/Copia de Benedetti.txt',
    '/content/drive/MyDrive/CURSO NLP/data/Copia de Neruda.txt'
]

In [ ]:
textos = []
etiquetas = []

for etiqueta, f in enumerate(archivos):
    print(f"{f} Corresponde a {etiqueta}")

    with open(f, 'r', encoding='utf-8') as archivo:
        for line in archivo:
            print(line)
            line = line.rstrip().lower()
            print(line)
            if line:
                # eliminar puntuación
                line = line.translate(str.maketrans('', '', string.punctuation))
                textos.append(line)
                etiquetas.append(etiqueta)
            print(line)


In [ ]:
textos

In [ ]:
etiquetas

In [ ]:
plt.hist(etiquetas)

In [ ]:
print(len(textos))
print(len(etiquetas))

In [ ]:
etiquetas = np.array(etiquetas)
etiquetas

In [ ]:
train_text, test_text, Ytrain, Ytest = train_test_split(textos, etiquetas, test_size = 0.1, random_state = 42)

In [ ]:
print(f'El modelo sera entrenado con {len(Ytrain)} y  probado con {len(Ytest)}')

In [ ]:
'''
unk es una conversion que se utiliza a menudo en PLN para representar palabras
desconocidas o fuera del vocabulario. En este caso, se esta asignando el indice
0 a la palabra especial.
'''
indice = 1
indice_palabras = {'unk': 0}

In [ ]:
# creamos diccionario de codificacion de palabras a indices
for texto in train_text:
    tokens = texto.split()
    for token in tokens:
        if token not in indice_palabras:
            indice_palabras[token] = indice
            indice += 1

In [ ]:
indice_palabras

In [ ]:
len(indice_palabras)

In [ ]:
train_text_int = []
test_text_int = []

for texto in train_text:
    tokens = texto.split()
    linea_entero = [indice_palabras[token] for token in tokens]
    train_text_int.append(linea_entero)

In [ ]:
train_text_int

In [ ]:
for texto in train_text:
  tokens = texto.split()
  line_as_int = [indice_palabras.get(token,0) for token in tokens]
  test_text_int.append(line_as_int)

In [ ]:
test_text_int

In [ ]:
test_text_int

In [ ]:
V = len(indice_palabras)
V

In [ ]:
A0 = np.ones((V,V))
pi0 = np.ones(V)


A1 = np.ones((V,V))
pi1 = np.ones(V)

In [ ]:
def compute_counts(text_as_int, A, pi):
    for tokens in text_as_int:
        last_idx = None
        for idx in tokens:
            # estamos en la primera palabra de la secuencia
            if last_idx is None:
                pi[idx] += 1
            else:
                A[last_idx, idx] += 1
            last_idx = idx

compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [ ]:
pi0

In [ ]:
pi1

In [ ]:
A0

In [ ]:
A1

Normalizar A y pi para que sean matrices de probabilidad validas

In [ ]:
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [ ]:
pi0

In [ ]:
A0

In [ ]:
# Espacio logarítmico, se usa ya que las probabilidades son muy pequeñas y el sistema trunque las probabilidades como 0 (no probable)
# Entonces, se usa para evitar que el modelo se muera computacionalmente
logA0 = np.log(A0)
logpi0 = np.log(pi0)


logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [ ]:
logpi0

In [ ]:
logA0

In [ ]:
count0 = sum(y==0 for y in Ytrain) # Cuenta de etiquetas de clase 0 en Ytrain
count1 = sum(y==1 for y in Ytrain) # Cuenta de etiquetas de clase 1 en Ytrain

total = len(Ytrain) # cantidad total de ejemplos de entrenamiento

p0 = count0 / total # probabilidad a priori de clase 0
p1 = count1 / total # probabilidad a priori de clase 1

logp0 = np.log(p0)  # Logaritmo de la probabilidad a priori de clase 0
logp1 = np.log(p1)  # Logaritmo de la probabilidad a priori de clase 1

print(f'Las probabilidades a priori de la clase 0 y 1 son respectivamente {p0, p1}')

# Construcción de un clasificador

In [ ]:
# contrucción de un clasificador
class Classifier:
    def __init__(self, logAs, logpis, logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors) # número de clases

    def _compute_log_likelihood(self, input_, class_):
        logA = self.logAs[class_]
        logpi = self.logpis[class_]

        last_idx = None
        logprob = 0
        for idx in input_:
            if last_idx is None:
                # Es el primer token en la secuencia
                logprob += logpi[idx]
            else:
                # Calcula la probabilidad de transición de la palabra anterior a la actual
                logprob += logA[last_idx, idx]

            # Actualiza last_idx para la próxima iteración
            last_idx = idx

        return logprob

    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        for i, input_ in enumerate(inputs):
            # Calcula los logaritmos de las probabilidades posteriores para cada clase
            posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
                          for c in range(self.K)]
            # Elige la clase con la mayor probabilidad posterior como la predicción
            pred = np.argmax(posteriors)
            predictions[i] = pred
        return predictions

In [ ]:
# Cada arreglo debe estar en orden ya que se asuma que las clases indexan estas listas
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [ ]:
Ptrain = clf.predict(train_text_int)

In [ ]:
print(f'Train acc: {np.mean(Ptrain==Ytrain)}')

In [ ]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")